In [7]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Import Packages

In [1]:
import platform

print("Running on Windows")
#path_main_folder = "/content/drive/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"
path_main_folder = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


path_data_folder = path_main_folder + "AIST_Cleaned/data3_san_stefano/"#"AIST_Cleaned/data2_mosheerIsmail/",data3_san_stefano
path_cfg = path_data_folder +"cfg.sumocfg"
path_project_folder = path_main_folder + ""



Running on Windows


In [2]:
import os
os.path.exists(path_project_folder)

True

In [3]:
import sys
sys.path.append(path_project_folder)

import Connections.ProtoRealConnection
from Connections.Connection import *
import os
import gymnasium as gym
from numpy import inf
import numpy as np
import SumoEnvSingleAgent
from Utils_reporting import *
from Utils_running_singleAgent import *
from rewards import *
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch
import time
from Callbacks import *
from models.d3qn import D3QNAgent

pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Load Parameters

In [4]:
durations = [7 , 15 ,20 ,30] #[15,30,60,90]
ENV_NAME = "HighGroupedSumoEnv"
REWARD_TYPE = "proposed_reward"
max_steps = 100000000 
max_sumo_steps = 2000
sumo_traffic_scale = 1
enable_variation_action = True
seed = 0



In [5]:
agent_id ='6082411793'
traffic_lights = [(4, agent_id)]


In [6]:
reward_func = {
'proposed_reward':reward_proposed,
'literature':reward_liter,
'project_reward':reward_proj,
}
env_classes = {
    "HighGroupedSumoEnv": SumoEnvSingleAgent.HighGroupedSumoEnv,
    "GroupedSumoEnv": SumoEnvSingleAgent.GroupedSumoEnv ,
    "SumoEnv": SumoEnvSingleAgent.SumoEnv,
}

In [7]:
def create_env(config_):
    args = (traffic_lights[0][1],traffic_lights[0][0], durations,reward_func[REWARD_TYPE], max_steps, max_sumo_steps,sumo_traffic_scale,enable_variation_action,config_,seed)
    env = env_classes[ENV_NAME](*args)
    return env



In [8]:
video_path = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/try/vid.mp4"
video_output_path = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/try/vid_out.mp4"
#yolo_path = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/try/pruned_yolo_model.pt"
yolo_path = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/try/yolo11n.pt"

thresould_speed = .01 #10 ,36 is very wrong as m/s #.01 m/s
yellow_seconds = 15

In [9]:
video_writer = None
speedestimator = None
screen = None
cap = None
DISPLAY_WIDTH = 800
DISPLAY_HEIGHT = 450


In [10]:
port = 'COM3'

## Make env

In [11]:
import importlib
import Connections.ProtoRealConnection
importlib.reload(Connections.ProtoRealConnection)


<module 'Connections.ProtoRealConnection' from 'i:\\My Drive\\study\\graduation_project\\final\\Code\\project_files\\TrafficManager\\TrafficManager\\Connections\\ProtoRealConnection.py'>

In [47]:
conn = Connections.ProtoRealConnection.ProtoRealConnection(port,video_path,yolo_path,yellow_seconds,thresould_speed = thresould_speed)

set_global_conn(conn)


In [13]:
model_loaded = PPO.load("models/high_Env_proposed_area3_test.zip")

In [14]:
env = create_env({})  # Create the environment instance
vec_env = DummyVecEnv([lambda: env])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=False)


env.reset()


Environment reset


(array([          0,           0,           0,           0,           0,           0,           0], dtype=float32),
 {})

In [15]:
conn.getCurrentState(agent_id)

array([          0,           0,           0,           0,           0,           0,           0])

In [16]:
from collections import deque
last_3_signals = deque(maxlen = 3)#3  

In [17]:
def ensure_queue_from_action(real_action):
    if (enable_variation_action):
        if (env._all_equal(last_3_signals)):
                real_action = conn.inverse_action(real_action)
                last_3_signals.append(real_action)
                return real_action
    
    last_3_signals.append(real_action)
    return real_action

In [ ]:
for i in range(6):
#while (conn._ret):
    # get state action
    state = conn.getCurrentState(agent_id)
    action, _states = model_loaded.predict(state, deterministic=False)
    real_action,real_duration = env.get_real_action(int(action))
    reward = reward_func[REWARD_TYPE](agent_id,state,real_action,real_action)

    # for better performance
    #real_duration = real_duration//10
    real_action = ensure_queue_from_action(real_action)
    print(real_action)
    
    # backend
    record_state_to_server(state,real_action,reward)
    
    # execute the action
    print(f"state is {state}  ,action {action} = {real_action,real_duration}")
    #env.step(int(action))
    #vec_env.step([int(action)])
    conn.do_step_one_agent(agent_id, real_action, real_duration, 500, 5)


In [31]:
env.get_real_action(int(action))

('gggg', 15)

In [28]:
action

array(5, dtype=int64)

In [26]:
env.last_3_signals

deque(['rrrr', 'gggg', 'gggg'], maxlen=3)

In [46]:
conn.ardinuo.close_arduino()

In [146]:
int(action)

6

In [23]:
import pymysql
connection = pymysql.connect(host="localhost",user="root", password="", database='system_state' )
cursor = connection.cursor()

def record_state_to_server(single_state,action,reward):
    # for those rewards last action is not used so just leave it action

    data = list(single_state) + [reward,action, action]
    
    placeholders = ", ".join(["%s"] * len(data))

    insert_query = f"INSERT INTO states1 VALUES ({placeholders})"

    cursor.execute(insert_query,data)
    
    connection.commit()
    #connection.close()
    
record_state_to_server(state,action,8.3)

In [ ]:
## Direction of speed??????? crop video ,as lane
## Real video /other video??
## set max to be increasing
## d3qn try? clean with env step?ensure always yellow in middle 
## backend

## Others

In [ ]:
conn.lane_video_estimatior.init_pygame()

In [ ]:
from multiprocessing import Process, Queue

def thread_yolo(frame):
    estimate = conn.lane_video_estimatior.speedestimator(frame)
    frame = estimate.plot_im
    spd = estimate.spd  # extract the speed value
    print("thread ended after", str(time.time() - start))
    return frame, spd
    
def yolo_worker(in_q, out_q):
    while True:
        frame = in_q.get()
        if frame is None:
            break
        result = thread_yolo(frame)    # do the heavy work
        out_q.put(result)


# in your setup code, once:
in_q, out_q = Queue(), Queue()
p = Process(target=yolo_worker, args=(in_q, out_q))
p.daemon = True
p.start()

# in your main loop:
for i in range(50):
    ret, frame = conn.lane_video_estimatior.cap.read()
    if not ret: break

    if i == 0:
        print("started main loop")

    # send frame off to the worker, non-blocking
    in_q.put(frame)

    # render immediately; if there's a result ready, grab it
    if not out_q.empty():
        yolo_result = out_q.get()
        frame, spd = yolo_result

    conn.lane_video_estimatior.render(frame)


started main loop


In [ ]:
conn.lane_video_estimatior.intialize_Model_CV()

In [61]:
# Just to verify
frame = conn.lane_video_estimatior.cap.read()[1]
estimate = conn.lane_video_estimatior.speedestimator(frame)
print("Estimate speed:", conn.lane_video_estimatior.speedestimator.spd)


Estimate speed: {}


In [ ]:
import threading
import queue

in_q = queue.Queue()
out_q = queue.Queue()

def yolo_worker():
    while True:
        frame = in_q.get()
        if frame is None:
            break
        annotated = run_yolo(frame)  # only detection, not speed
        out_q.put(annotated)

# Start thread once
t = threading.Thread(target=yolo_worker, daemon=True)
t.start()

# Main loop
for i in range(50):
    ret, frame = conn.lane_video_estimatior.cap.read()
    if not ret:
        break

    # Always run speed estimation
    estimate = conn.lane_video_estimatior.speedestimator(frame)
    speed = estimate.spd

    # Every N frames, send frame to background thread for YOLO
    if i % 5 == 0:
        if not in_q.full():
            in_q.put(frame)

    # Draw detection result if ready
    if not out_q.empty():
        detection_frame = out_q.get()
        conn.lane_video_estimatior.render(detection_frame)
    else:
        conn.lane_video_estimatior.render(estimate.plot_im)

    print(f"Speed: {speed:.2f}")


In [ ]:
conn.lane_video_estimatior.speedestimator.spd

{}

In [ ]:
print(conn.getCurrentState(0))

[          0           0           0           0           0           0           0]


In [ ]:
ه
frame =''
start = time.time()
for i in range(25):

    ret, frame = conn.lane_video_estimatior.cap.read()
    if not ret:
        print("end")
        break

    if (i ==0):
        print("thread started after",str(time.time()-start))
        th =  threading.Thread(target=thread_yolo, args=(frame.copy(),))
        print("thread started after",str(time.time()-start))
        th.start()

    conn.lane_video_estimatior.render(frame)
    print("frame ",str(i),"end ",str(time.time()-start))
end = time.time()
print(end-start)

thread started after 0.03598952293395996


NameError: name 'threading' is not defined

In [ ]:
(1/25)*2515

0.6

In [ ]:
conn.update_video('g',1,25,estimate)

In [ ]:
conn.ardinuo.close()

NameError: name 'conn' is not defined

In [ ]:
model_loaded = PPO.load("models/high_Env_proposed_area3_test.zip")

In [ ]:
env = create_env({})  # Create the environment instance
vec_env = DummyVecEnv([lambda: env])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=False)


env.reset()
## Direction of speed??????? crop video ,as lane
## Real video /other video??

Environment reset


(array([          0,           0,           0,           0,           0,           0,           0], dtype=float32),
 {})

In [ ]:
## Direction of speed??????? crop video ,as lane
## Real video /other video??
## thresould may be .1 as sumo
## as sumo waiting time is accumalted ,even in signal
## modify waiting ,to not forget ? ,or remove only them at get ???
## solution to time processing problem

In [ ]:
### Need test and revision
last_real_action = 'r'
frame = None
ret = None
ret,frame = conn.update_video('g',1,25,estimate)
yellow_time  = 2
ardinuo_time = .5
for i in range(7):
    # next for one step (equal duration)
            if not ret :
                    print("end")
                    break


            #if (use_model):


            state = conn.getCurrentState(agent_id)
            action, _states = model_loaded.predict(state, deterministic=False)
            real_action,real_duration = env.get_real_action(int(action))
            real_duration = real_duration /4
            print(f"state is {state}  ,action {action} = {real_action,real_duration}")


            #pygame.time.delay(1)
            #use_model = False #---------> real_duration * fps
            action_lane_1 = real_action[0]


            # perform action
            time_to_apply = real_duration // conn.lane_video_estimatior.one_frame_processing

            if last_real_action == action_lane_1:
                start = time.time()
                conn.ardinuo.send_command(arduino, action_lane_1,one_second_processing*time_to_apply)
                ret,frame = update_video(action_lane_1,time_to_apply,fps,ret,frame)
                end =time.time()
                time.sleep (1+ one_second_processing*time_to_apply -(end-start))
                print(f"action {action_lane_1} is ended after {end-start}")

            else:
                start =time.time()
                send_command(arduino, 'y',one_second_processing*yellow_time)
                ret,frame = update_video('y',yellow_time,fps,ret,frame)
                end =time.time()
                time.sleep (1+one_second_processing*time_to_apply-(end-start))
                print(f"action y is ended after {end-start}")

                start =time.time()
                send_command(arduino, action_lane_1,one_second_processing*time_to_apply)
                ret,frame = update_video(action_lane_1,time_to_apply,fps,ret,frame)
                end =time.time()
                time.sleep (1+one_second_processing*time_to_apply-(end-start))
                print(f"action {action_lane_1} is ended after {end-start}")


            last_real_action = action_lane_1


state is [      14.63      127.09      11.044      3.0277         225           9           1]  ,action 5 = ('gggg', 30)
[      12.12       12.12       12.12       12.12       12.12       12.12       10.96        7.96        7.76]
8.868475437164307
state is [      13.25      131.51       10.92      14.104         250          10           1]  ,action 0 = ('rrrr', 15)
[      13.12       13.12       13.12       13.12       13.12       13.12       11.96        8.96        8.76         0.8]
10.006856441497803
state is [      13.25      131.51       11.92      14.104         250          10           1]  ,action 7 = ('gggg', 90)
[      14.12       14.12       14.12       14.12       14.12       14.12       12.96        9.96        9.76         1.8]


KeyboardInterrupt: 

In [ ]:
last_duration_fps = 0 #duration * fps
last_real_action = 'g'

for i in range(27):
    # next for one step (equal duration)

    if (last_real_action == 'g'):
            ret, frame_vid = cap.read()
            if not ret:
                print("end")
                break
            estimate = speedestimator(frame_vid)
            frame = estimate.plot_im
            render(frame)
    else:
            render(frame)
            time.sleep(1)

    update_waiting_video(speedestimator,fps)

    if (last_duration_fps==0):


            state = get_State_video(speedestimator,fps)
            action, _states = model_loaded.predict(state, deterministic=False)
            real_action,real_duration=env.get_real_action(int(action))
            print(f"state is {state}  ,action {action} = {real_action,real_duration}")


            #pygame.time.delay(1)
            last_duration = real_duration
            last_duration_fps = 5 #---------> real_duration * fps
            last_real_action = real_action[0]
            #send_command(arduino, last_real_action,last_duration_fps)
    else :
            last_duration_fps -= 1



state is [     11.175       145.8         0.2  7.7037e-34         325          13           1]  ,action 3 = ('rrrr', 90)
state is [     11.175       145.8        0.24  3.0815e-33         325          13           1]  ,action 4 = ('gggg', 15)
state is [     11.175       145.8        0.24  3.0815e-33         325          13           1]  ,action 3 = ('rrrr', 90)
state is [     11.175       145.8        0.24  3.0815e-33         325          13           1]  ,action 7 = ('gggg', 90)
state is [     11.175       145.8        0.24  3.0815e-33         325          13           1]  ,action 1 = ('rrrr', 30)


In [ ]:
intial_state = np.array([7]*7)

In [ ]:
for i in range(10):
    action, _states = model_loaded.predict(10*np.random.random((1,7)), deterministic=True)
    print("action",action)

action [1]
action [3]
action [3]
action [1]
action [1]
action [0]
action [0]
action [3]
action [1]
action [1]


In [ ]:
action, _states = model_loaded.predict(np.array([5]*7), deterministic=True)
action = int(action)

In [ ]:
real_action,real_duration=env.get_real_action(int(action))
real_action,real_duration

('rrrr', 2)

In [ ]:
#conn2.state = {agent_id:np.array([5]*7)}

In [ ]:
obs, reward, done,terminated, info = env.step(int(action))


Progress: 1/2000, Sumo Time 0, state [0 0 0 0 0 0 0], action gggg, duration 2, reward -0.6666666666666666

In [ ]:
env.encoded_action_mapping

{0: ('rrrr', 4),
 1: ('rrrr', 2),
 2: ('rrrr', 3),
 3: ('rrrr', 5),
 4: ('gggg', 4),
 5: ('gggg', 2),
 6: ('gggg', 3),
 7: ('gggg', 5)}

In [ ]:
import serial
import time
def set_Esp32(port):
    # Replace with your actual COM port. For example, '/dev/ttyUSB0' on Linux or 'COM3' on Windows.
    arduino = serial.Serial(port, 115200 )
    time.sleep(2)  # Give Arduino time to reset
    return arduino

def send_command(arduino, light, duration):
    # Create the command string
    command = f"{light},{duration}\n"  # Format: light,duration
    # Send the command to the Arduino
    arduino.write(command.encode())
    time.sleep(0.5)  # Wait for a short period to ensure the command is sent


def close_arduino(arduino):
    # Close the serial connection
    arduino.close()





In [ ]:
arduino = set_Esp32('COM3')

SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)

In [ ]:
## single light only<-------- ,may later add the other lights by access last index as duration
#send_command(arduino, real_action[0],real_duration)

#### # Test the connection and sending commands
import random
for i in range(4):
    #while True:
    real_action = random.choice(['g','r','y'])
    real_duration = random.choice([5,10])
    print("action",real_action,"duration",real_duration)
    send_command(arduino, real_action,real_duration)
    time.sleep(real_duration+1) # Wait for the duration of the light  

In [ ]:
for i in range(4):
    #while True:
    action, _states = model_loaded.predict(np.array([5]*7), deterministic=False)
    real_action,real_duration=env.get_real_action(int(action))
    print("action",real_action,"duration",real_duration)
    send_command(arduino, real_action[0],real_duration)
    time.sleep(real_duration)

action gggg duration 2
action rrrr duration 2
action rrrr duration 5
action gggg duration 5


In [ ]:
close_arduino(arduino)